# PDF CV

https://ohmycv.app/edit/1715014690945

In [1]:
# read yaml file
import yaml

with open('../_data/data.yml') as f:
    data = yaml.load(f, Loader=yaml.FullLoader)

In [2]:
# import md template
template = open('template_cv.md', 'r').read()
# Remove spaces from all {{ }}
template = template.replace("{{ ", "{{").replace(" }}", "}}")

In [3]:
# Base info

def replace_all(template, data, base_keys=[]):
    for key in data:
        if isinstance(data[key], dict):
            # print("Recursing into", key)
            template = replace_all(template, data[key], base_keys + [key])
        else:
            key_str = '.'.join(base_keys + [key])
            # print("Replacing", key_str)
            template = template.replace('{{' + key_str + '}}', str(data[key]))
    return template

cv = replace_all(template, data)

In [4]:
# Fill for all elements 

def loop_fill(template, data, cv=""):
    md = ""
    for key in data:
        md += replace_all(template, key)
    return md

In [5]:
# Education

template = """
**{{degree}}**
~ {{university}}, Toulouse, France
~ {{time}}
"""

md = loop_fill(template, data['education']["info"])
# Add PhD
md = """
**PhD in Machine Learning**
~ ISAE-SUPAERO, Toulouse, France
~ 2021-2024

- Topic: Leveraging Structures in Evolutionary Neural Policy Search
- Supervisors: Emmanuel Rachelson, Dennis Wilson.
""" + md


cv = cv.replace('{{education}}', md)


In [6]:
# Work
template = """
**{{role}}**
~ {{company}}
~ {{time}}

- {{details}}
"""

xp_data = data['experiences']["info"]
for xp in xp_data:
    # print(xp)
    xp['supervisor'] = ""
    if "supervised" in xp:
        xp['supervisor'] = "\nSupervised by " + " ".join(s["name"] for s in xp['supervised'])
    if "details" in xp:
        # remove last \n
        xp['details'] = xp['details'][:-1]
        xp['details'] = xp['details'].replace("\n", "\n- ")



md = loop_fill(template, xp_data)
# print(md)
cv = cv.replace('{{experiences}}', md)

In [7]:
# Publications
template = """
**{{title}}**

~ {{conference}}

- {{authors}} {{link}} {{code}} {{blog}}

"""

publi_data = data['publications']["research"]
for publi in publi_data:
    if "link" in publi:
        publi['link'] = f"[(paper)]({publi['link']})"
    else:
        publi['link'] = "(paper soon)"
    if "code" in publi:
        publi['code'] = f" [(code)]({publi['code']})"
    else:
        publi['code'] = ""
    if "blog" in publi:
        publi['blog'] = f" [(blog)]({publi['blog']})"
    else:
        publi['blog'] = ""

md = loop_fill(template, publi_data)
# print(md)
cv = cv.replace('{{publications}}', md)


In [8]:
# write to file
with open('cv.md', 'w') as f:
    f.write(cv)